In [31]:
# análise completa com todas as instancias do dataset
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # Treinando o modelo

# Exibe as classes identificadas pelo modelo
classes = modelo.classes_
print(f"Classes identificadas pelo modelo: {classes}")
print(f"Nomes das classes: {iris['target_names']}")

# Informações sobre o dataset
num_instancias = len(X_test)
print(f"\nNúmero total de instâncias no conjunto de teste: {num_instancias}")

# Escolher qual instância analisar ou todas
# Escolha da instância específica (por exemplo, 0) ou `None` para todas
instancia_para_analisar = 0  
# Operador ternàrio - se a instancia for (None) salva todas na variavel, se não adiciona somente a instância escolhida
instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
print(instancia_para_analisar)
# Loop para analisar instâncias
for idx in instancias_para_analisar:
    Vs = X_test.iloc[idx].to_dict()  # Utilizando a instância de teste especificada
    instancia_test = X_test.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

    # Previsão de probabilidades para a instância
    probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe usando o 0 para acesar a 1ª linha

    # O valor de gamma_A é a probabilidade da (classe verdadeira - é o rotulo original do dataset o qual pertence a instância)
    classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
    gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira treinada pela reg.logistica na func predict_proba

    # Exibe a classe verdadeira e as probabilidades
    print(f"\nInstância {idx}:")
    print(f"Classe verdadeira: {classe_verdadeira} ({iris['target_names'][classe_verdadeira]})")
    print(f"Probabilidades: {probs}")
    print(f"Valor de gamma_A: {gamma_A}")

    # Cálculo dos deltas para cada feature
    delta = []
    w = modelo.coef_[0]  # Pesos do modelo treinado

    for i, feature in enumerate(df.columns[-1]):
        if w[i] < 0: # se o peso for negativo
            delta.append((Vs[feature] - df[feature].min()) * w[i])
        else:
            delta.append((df[feature].max() - Vs[feature]) * w[i])

    # Calcula R como a soma dos deltas menos gamma_A   #### no artigo não menciona se o R pode ou não ser negativo, R negativo sugere que as características                                         não estão explicando suficientemente a predição
    
    R = sum(delta) - gamma_A
    #R = abs(sum(delta) - gamma_A) # a função abs alem de tratar o valor absoluto permite sempre o resultado ser positivo ou seja ela remove o simbolo negativo, porem acredito que está calculando errado

    #R = max(0, sum(delta) - gamma_A)

   
    # Computa a PI-explicação
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0

    # Calcula a explicação
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = X_test.columns[sorted_idx]  # Nome da feature correspondente
        feature_value = Vs[feature]  # Valor da feature para a instância

        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")

        R_atual -= delta_value
        Idx += 1

    # Imprime a PI-explicação
    print(f"Valor de R: {R}")
    print("PI-Explicação:")
    for item in Xpl:
        print(f"- {item}")


Classes identificadas pelo modelo: [0 1 2]
Nomes das classes: ['setosa' 'versicolor' 'virginica']

Número total de instâncias no conjunto de teste: 30
0

Instância 0:
Classe verdadeira: 1 (versicolor)
Probabilidades: [0.00380009 0.82771461 0.16848529]
Valor de gamma_A: 0.8277146135745481


KeyError: 't'